<a href="https://colab.research.google.com/github/Seph-iroth/RoboLearning/blob/main/mecs6616_Spring2024_Project2_JL6080.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MECS6616 Spring 2024 - Project 2**

# **Introduction**

***IMPORTANT:***
- **Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/197115/pages/assignment-instructions) page on Courseworks to understand the workflow and submission requirements for this project.**

This project aims to demonstrate how neural networks can be used in a robotics setting. We will continue using the 2D maze environment introduced in Project 1 and learn to navigate an agent to a goal. Since neural networks can be more powerful models than the ones we had access to in Project 1, we can afford to make some changes to the 2D maze environment and make the problem more difficult. The project is divided into three parts: In Part I, you will train a simple Deep Neural Network (DNN) to predict the optimal action towards the goal given the agent position and the goal position. In Parts II and III, you will train Convolutional Neural Networks (CNNs) to predict the optimal action given images of the maze environment.

<div>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/P1_side.png?raw=true" width="300"/>
</div>

The figure above illustrates the simulation world, where the "robot" (also referred to as "agent") is represented by a green dot, and the goal location is marked by a red square. The agent's objective is to navigate to this goal location, avoiding any obstacles (depicted as black boxes) along the way.

**Unlike the previous project, the robot and the goal are spawned at random positions in the maze.** Also, the action space now contains all four directions: 'up', 'down', 'left' and 'right'. Another change is that, in addition to the obstacle map shown above, we introduce two new obstacle maps as shown below. However, these new maps will not be used until Part III.

<div>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/map1.png?raw=true" width="300"/>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/map2.png?raw=true" width="300"/>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/map3.png?raw=true" width="300"/>
</div>

We want to learn to navigate the agent by imitating demonstrations from an expert user. In all three parts, you will be using data collected by a human controlling the agent via a keyboard for training.

# **Project Setup (do NOT change)**

***IMPORTANT:***
- Do NOT change this "*Project Setup*" section
- Do NOT install any other dependencies or a different version of an already provided package. You may, however, import other packages
- Your code should go under the subsequent sections with headings "*Part 1*", "*Part 2*", and "*Part 3*"
- You may find it useful to minimize sections using the arrows located to the left of each section heading

You will be accessing data files located in a Google Drive folder. The following cell downloads the data from the cloud

In [ ]:
# DO NOT CHANGE
# Download data
!wget https://www.dropbox.com/scl/fi/slf93ry48wroksempi6or/project2.zip?rlkey=pw9m14sm98ga1259crst3ss5l&dl=0
!mv project2.zip?rlkey=pw9m14sm98ga1259crst3ss5l project2.zip

--2024-02-27 05:31:52--  https://www.dropbox.com/scl/fi/slf93ry48wroksempi6or/project2.zip?rlkey=pw9m14sm98ga1259crst3ss5l
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9594dbedde6580d4c8a0543b60.dl.dropboxusercontent.com/cd/0/inline/COB2V3j4DhEqo6QoizTDG-8XnXgBmv3ldAWgJ3Pi15WxZxOpQ-odXvV7ake7_xIt8Nv0ZjXSSwXNRMQIgTRcOtZpD0eVBsaoQFtZalK3rQPgGNN9Qkte2IWJ4dT9q-UPOwF0KTLjn76_gBzQ66KVYFzm/file# [following]
--2024-02-27 05:31:54--  https://uc9594dbedde6580d4c8a0543b60.dl.dropboxusercontent.com/cd/0/inline/COB2V3j4DhEqo6QoizTDG-8XnXgBmv3ldAWgJ3Pi15WxZxOpQ-odXvV7ake7_xIt8Nv0ZjXSSwXNRMQIgTRcOtZpD0eVBsaoQFtZalK3rQPgGNN9Qkte2IWJ4dT9q-UPOwF0KTLjn76_gBzQ66KVYFzm/file
Resolving uc9594dbedde6580d4c8a0543b60.dl.dropboxusercontent.com (uc9594dbedde6580d4c8a0543b60.dl.dropboxusercontent.com)... 162.125.81.15, 262

In [ ]:
# Make sure you have successfully uploaded the zip file to Colab before running the line below.
# If wget fails to pull the zip file, you can download the zipfile from dropbox and manually upload it to collab instead
# If you do decide to manually upload the file, use the dropbox link in the previous cell (after wget) to access the file
# Make sure the zip file is named "project2.zip", rename it before uploading (if necessary)
# Upload the entire zip file to google colab. Do not unzip before uploading

# Unzip the uploaded zip file
!unzip project2.zip -d /content/
!mv project2/* /content/

Archive:  project2.zip
   creating: /content/project2/data/
  inflating: /content/project2/data/all_maps.pkl  
  inflating: /content/project2/data/map1.pkl  
  inflating: /content/project2/data_utils.py  
  inflating: /content/project2/dnn.py  
   creating: /content/project2/imgs/
  inflating: /content/project2/imgs/map1.png  
  inflating: /content/project2/imgs/map2.png  
  inflating: /content/project2/imgs/map3.png  
  inflating: /content/project2/imgs/P1_side.png  
   creating: /content/project2/mjcf/
   creating: /content/project2/mjcf/common/
  inflating: /content/project2/mjcf/common/materials.xml  
  inflating: /content/project2/mjcf/common/skybox.xml  
  inflating: /content/project2/mjcf/common/visual.xml  
  inflating: /content/project2/mjcf/point_mass.xml  
  inflating: /content/project2/mjcf/test_mjcf.xml  
  inflating: /content/project2/score_policy.py  
  inflating: /content/project2/simple_maze.py  


In [ ]:
# DO NOT CHANGE

# Install required packages
!pip install pybullet numpngw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 9.3 MB/s eta 0:00:00


# Part I. Behavioral cloning with low dimensional data

This part is a natural extension of Part II in Project 1, where your agent needs to learn a policy using labeled examples from an expert.

Each labeled example $i$ will contain a tuple of the form $(o, a)^i$, where $o$ represents an observation and $a$ represents the action taken by the expert given that observation. You must simply learn to imitate the expert, a process also known as behavioral cloning. Note that while the observation space will be different in each part, the action space is the same for the rest of the project.

We will be training a DNN policy to predict an action to be taken ('up', 'down', 'left', and 'right') based on the observation. **In Part I, the observation will contain the agent position and the current goal position.** (Since the goal is sampled randomly, the policy has to know the current goal to be reached). The environment thus returns an observation array of size (4, ) where the agent position is contained in the first two axes and the current goal position is contained in the next two. **In Part I, the map that the robot is navigating is always the same.**

PyTorch and Tensorflow are two popular frameworks for building and training neural networks but for this class, we will be exclusively using PyTorch and you are allowed to use any of its features. A good starting point can be found [here](https://github.com/roamlab/robot-learning-S2024/blob/main/dnn_example.py).

You will implement a class that inherits from `RobotPolicy` by providing implementations for the abstract methods from the class. These abstract methods will be re-used by future parts of the project, so do not edit them.



In [ ]:
# DO NOT CHANGE
# base class

import abc


class RobotPolicy(abc.ABC):

    @abc.abstractmethod
    def train(self, data):
        """
            Abstract method for training a policy.

            Args:
                data: a dict that contains X (key = 'obs') and y (key = 'actions').

                X is either rgb image (N, 64, 64, 3) OR  agent & goal pos (N, 4)

            Returns:
                This method does not return anything. It will just need to update the
                property of a RobotPolicy instance.
        """

    @abc.abstractmethod
    def get_action(self, obs):
        """
            Abstract method for getting action. You can do data preprocessing and feed
            forward of your trained model here.
            Args:
                obs: an observation (64 x 64 x 3) rgb image OR (4, ) positions

            Returns:
                action: an integer between 0 to 3
        """

In [ ]:
# Implement your solution for Part 1 below
import torch
import torch.nn as nn
import torch.optim as optim

class DNNPolicy(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        torch.manual_seed(42)
        super(DNNPolicy, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        return self.network(x)

class POSBCRobot:  # Assuming RobotPolicy is defined elsewhere or not needed
    def __init__(self):
        torch.manual_seed(42)
        self.input_size = 4  # Size of the observation
        self.hidden_size = 256  # Arbitrary choice, can be tuned
        self.output_size = 4  # Actions: 'up', 'down', 'left', 'right'
        self.policy = DNNPolicy(self.input_size, self.hidden_size, self.output_size)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=0.01)
        self.criterion = nn.CrossEntropyLoss()

    def train(self, data):
        torch.manual_seed(42)
        observations = torch.tensor(data['obs'], dtype=torch.float)
        actions = torch.tensor(data['actions'], dtype=torch.long)

        for _ in range(700):
            self.optimizer.zero_grad()
            loss = self.criterion(self.policy(observations), actions)
            loss.backward()
            self.optimizer.step()

    def get_action(self, obs):
        with torch.no_grad():
            obs = torch.tensor(obs, dtype=torch.float).unsqueeze(0)
            action = torch.argmax(self.policy(obs), dim=1).item()
        return action
# for i in range(10):
#   score1 = score_policy.score_pos_bc(policy=POSBCRobot(), gui=False, model=None)
#   grade1 = score1 / part1_bound * 4 if score1 < part1_bound else 4

#   print('\n---')
#   print(f'Part 1 Score: {score1}')
#   print(f'Part 1 Grade: {score1:.2f} / {part1_bound:.2f} * 4 = {grade1:.2f}')

## Evaluation and Grading

We will evaluate your model by simply having the agent follow the commands that it provides.  We will evaluate for 100 different randomly sampled starting positions and goals. For each goal, we roll out the trained policy for 50 steps. After the 50 steps, we will evaluate the closest distance to the goal the agent has ended up. If the agent reaches < 0.1 distance from the goal, the episode is ended before 50 steps and the minimum distance will be recorded as 0. The score is the fraction of the initial distance to goal covered by the agent averaged over 100 trials. Your final grade will be computed based on this score.

We will calculate the score using the formula :

```score = avg[(init_dist -  min_dist) / init_dist]```

We will auto-generate your grades using the code below. The grading of each part is separate from each other so you can get the grade right after each part is finished.

The total points of this assignment are 15. According to the difficulty level of each part, parts 1, 2, and 3 have 4, 5, 6 points respectively.

- Part 1: if your score >= 0.99, you will receive 4 / 4. Otherwise, your final grade will be score / 0.99 * 4.
- Part 2: if your score >= 0.95, you will receive 5 / 5. Otherwise, your final grade will be score / 0.95 * 5.
- Part 3: if your score >= 0.95, you will receive 6 / 6. Otherwise, your final grade will be score / 0.95 * 6.

The score function for each part provides two extra arguments to assist your debugging.

- gui: If this is set to True, you will save the behavior of the agents during evaluation as an animation file. This animation file can be visualized using the provided code below to help you understand the behavior of the agent. **Please set it to False before your submission as it will slow down evaluation.**
- model: If you provide a path to a saved model, the score function will not train from scratch but will instead load the save model. **Please set it to None before submission.** Any models you generate during runtime will be automatically deleted when disconnected. The grader will train the model from scratch.

In [ ]:
# DO NOT CHANGE
# Set up grading

import score_policy
import importlib
importlib.reload(score_policy)
from IPython.display import Image


part1_bound = 0.99
part2_bound = 0.95
part3_bound = 0.95

In [ ]:
# DO NOT CHANGE
# Getting the score and grade for Part 1

score1 = score_policy.score_pos_bc(policy=POSBCRobot(), gui=False, model=None)
grade1 = score1 / part1_bound * 4 if score1 < part1_bound else 4

print('\n---')
print(f'Part 1 Score: {score1}')
print(f'Part 1 Grade: {score1:.2f} / {part1_bound:.2f} * 4 = {grade1:.2f}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



---
Part 1 Score: 0.9949476462423499
Part 1 Grade: 0.99 / 0.99 * 4 = 4.00


In [ ]:
# Optionally, uncomment and run the code below if you have saved an animation (gui = True) that you want to visualize.

# Image(filename='part_1_anim.png', width=200, height=200)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Part II. Behavioral cloning with visual observations

In this part, you are asked to do a similar task as Part I, **but the observations will be RGB image observations of the world**, similar to the ones you used to do localization in Part III of Project 1. To process the RGB images, you will be implementing a CNN using PyTorch. [The official PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) is a good starting point. As in Part I, the map that the robot is navigating is always the same. **This means that your model really only has to learn how to figure out where the robot and the goal are located, and how to navigate around a fixed set of obstacles.**

All requirements from your code, as well as the evaluation method, are unchanged compared to Part I. The only difference is the nature of the observation that is provided to you.

In [ ]:
# Implement your solution for Part 2 below
# import gc
# gc.collect()
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import time
import torch.utils.checkpoint as checkpoint

torch.manual_seed(42)

class CNN222(nn.Module):
    def __init__(self):
        super(CNN222, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)  # 3 input channels for RGB images
        #(N, C, H, W)
        #where N is the batch size, C is the number of channels, H is the image height and W is the image width.
        #You will need to switch axes for the incoming images in order for them to be correctly passed to the first convolution layer.
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 4)  # 4 output actions: 'up', 'down', 'left', 'right'

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = checkpoint.checkpoint(self.conv1_relu_pool, x)

        # x = checkpoint.checkpoint(self.conv2_relu_pool, x)
        x = x.reshape(-1, 32 * 16 * 16)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class RGBBCRobot1(RobotPolicy):
    def __init__(self):
        torch.manual_seed(42)
        super(RGBBCRobot1, self).__init__()
        self.policy = CNN222()
        self.optimizer = optim.Adam(self.policy.parameters(), lr=0.001, weight_decay=0.000001) #0.00001
        self.criterion = nn.CrossEntropyLoss()

    def train(self, data):
        print("here")
        torch.manual_seed(42)
        observations = torch.tensor(data['obs'], dtype=torch.float32)
        # Permute the dimensions of the observations to match (batch_size, channels, height, width)
        observations = observations.permute(0, 3, 1, 2)  # Moving the channel to the second dimension
        actions = torch.tensor(data['actions'], dtype=torch.long)

        dataset = TensorDataset(observations, actions)
        dataloader = DataLoader(dataset, batch_size=12, shuffle=True,num_workers=0)
        start_time = time.time()
        for epoch in range(9):  # Number of epochs
            for inputs, labels in dataloader:
                self.optimizer.zero_grad()
                outputs = self.policy(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')
        end_time = time.time()
        duration = end_time - start_time
        print(f"The code took {duration/60} minte to run.")
        print("Done")

    def get_action(self, obs):
        obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
        obs = obs.permute(0, 3, 1, 2)  # Adjust dimensions to match (batch_size, channels, height, width)
        self.policy.eval()
        with torch.no_grad():
            outputs = self.policy(obs)
            _, predicted = torch.max(outputs, 1)
            action = predicted.item()
        # print(action)
        return action

# for i in range(10):
#   start_time = time.time()
#   score2 = score_policy.score_rgb_bc1(policy=RGBBCRobot1(), gui=True, model=None)
#   grade2 = score2 / part2_bound * 5 if score2 < part2_bound else 5
#   end_time = time.time()
#   duration = end_time - start_time
#   print(f"The code took {duration/60} minte to run.")

#   print('\n---')
#   print(f'Part 2 Score: {score2}')
#   print(f'Part 2 Grade: {score2:.2f} / {part2_bound:.2f} * 5 = {grade2:.2f}')
# print('\n---------------------------')

## Evaluation and Grading

In [ ]:
# DO NOT CHANGE
# Getting the score and grade for Part 2

score2 = score_policy.score_rgb_bc1(policy=RGBBCRobot1(), gui=False, model=None)
grade2 = score2 / part2_bound * 5 if score2 < part2_bound else 5

print('\n---')
print(f'Part 2 Score: {score2}')
print(f'Part 2 Grade: {score2:.2f} / {part2_bound:.2f} * 5 = {grade2:.2f}')

here
Epoch 1, Loss: 0.8125088214874268
Epoch 2, Loss: 0.40781521797180176
Epoch 3, Loss: 0.5500264763832092
Epoch 4, Loss: 0.11845032870769501
Epoch 5, Loss: 0.015521619468927383
Epoch 6, Loss: 0.41534778475761414
Epoch 7, Loss: 0.0044367811642587185
Epoch 8, Loss: 0.02832048386335373
Epoch 9, Loss: 0.009282520972192287
The code took 2.733351457118988 minte to run.
Done

---
Part 2 Score: 0.9733892650564593
Part 2 Grade: 0.97 / 0.95 * 5 = 5.00


In [ ]:
# Optionally, uncomment and run the code below if you have saved an animation (gui = True) that you want to visualize.

# Image(filename='part_2_anim.png', width=200, height=200)

# Part III. Behavioral cloning with visual observations - multiple maps

This part is the same as  Part II except that it is trained and tested differently. **The training set involves expert demonstrations for the two new obstacle maps. And while testing, for each trial, a different obstacle map is randomly selected.** This means that your model has to learn how to reason about what an obstacle is, and how to go around it, based on nothing more than an image. The main objective of this part is to show that, when using a CNN, it is possible for a model to achieve this. The evaluation method for this part is the same as Part I and II.

In [ ]:
# Implement your solution for Part 3 below
#Part 3 Grade: 0.92 / 0.95 * 6 = 6
#------------------------------------------------------------------------------------------
# actions (12000,)
# obs (12000, 64, 64, 3)
# import gc
# gc.collect()
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class NBC(nn.Module):
    def __init__(self):
        super(NBC, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 16 * 16,128)  # With 32 feature maps and each of size 16x16 (after two pooling operations),
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 4)

        # self.fc2 = nn.Linear(128, 64)
        # self.fc2_2 = nn.Linear(64, 32)
        # self.fc3 = nn.Linear(32, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(-1, 32 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # x = F.relu(self.fc2_2(x))
        return self.fc3(x)


class RGBBCRobot2(RobotPolicy):
    def __init__(self):
        torch.manual_seed(42)
        super(RGBBCRobot2, self).__init__()
        self.policy = NBC()
        self.optimizer = optim.Adam(self.policy.parameters(), lr=0.0005,weight_decay=0.000001)
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    def train(self, data):

        observations = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2)  # (N, C, H, W)
        actions = torch.tensor(data['actions'], dtype=torch.long)

        dataset = TensorDataset(observations, actions)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True,num_workers=6,pin_memory=False)

        # print("start Trainning")
        for epoch in range(10):  # Training for 10 epochs
            for inputs, labels in dataloader:
                self.optimizer.zero_grad()
                outputs = self.policy(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

    def get_action(self, obs):
        obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2)  # (1, C, H, W)
        self.policy.eval()
        with torch.no_grad():
            outputs = self.policy(obs)
            _, predicted = torch.max(outputs, 1)
            action = predicted.item()
        return action

# # Implement your solution for Part 3 below
# #Part 3 Grade: 0.92 / 0.95 * 6 = 5.96
# #------------------------------------------------------------------------------------------
# # actions (12000,)
# # obs (12000, 64, 64, 3)
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset

# class NBC(nn.Module):
#     def __init__(self):
#         super(NBC, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(32 * 16 * 16,128)  # With 32 feature maps and each of size 16x16 (after two pooling operations),
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 4)

#         # self.fc2 = nn.Linear(128, 64)
#         # self.fc2_2 = nn.Linear(64, 32)
#         # self.fc3 = nn.Linear(32, 4)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.reshape(-1, 32 * 16 * 16)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))

#         # x = F.relu(self.fc2_2(x))
#         return self.fc3(x)


# class RGBBCRobot2(RobotPolicy):
#     def __init__(self):
#         torch.manual_seed(42)
#         super(RGBBCRobot2, self).__init__()
#         self.policy = NBC()
#         self.optimizer = optim.Adam(self.policy.parameters(), lr=0.0005,weight_decay=0.000001)
#         self.criterion = nn.CrossEntropyLoss()

#     def train(self, data):

#         observations = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2)  # (N, C, H, W)
#         actions = torch.tensor(data['actions'], dtype=torch.long)

#         dataset = TensorDataset(observations, actions)
#         dataloader = DataLoader(dataset, batch_size=64, shuffle=False,num_workers=6,pin_memory=False)

#         # print("start Trainning")
#         for epoch in range(2):  # Training for 9 epochs
#             for inputs, labels in dataloader:
#                 self.optimizer.zero_grad()
#                 outputs = self.policy(inputs)
#                 loss = self.criterion(outputs, labels)
#                 loss.backward()
#                 self.optimizer.step()
#             print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

#     def get_action(self, obs):
#         obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2)  # (1, C, H, W)
#         self.policy.eval()
#         with torch.no_grad():
#             outputs = self.policy(obs)
#             _, predicted = torch.max(outputs, 1)
#             action = predicted.item()
#         return action


def score(times):
  for i in range(times):
    start = time.time()
    part1_bound = 0.99
    part2_bound = 0.95
    part3_bound = 0.95
    score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(), gui=False, model=None)
    end = time.time()
    grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6

    print(f'Score: {score3}',f'Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}','it took',(end - start)/60,'min' )
    print('\n--------------------------------------------------------------------------------- ')
# score(2)

In [ ]:
# Implement your solution for Part 3 below
#Part 3 Grade: 0.92 / 0.95 * 6 = 6
#------------------------------------------------------------------------------------------
# actions (12000,)
# obs (12000, 64, 64, 3)
# import gc
# gc.collect()
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class NBC(nn.Module):
    def __init__(self):
        super(NBC, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 16 * 16,128)  # With 32 feature maps and each of size 16x16 (after two pooling operations),
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 4)

        # self.fc2 = nn.Linear(128, 64)
        # self.fc2_2 = nn.Linear(64, 32)
        # self.fc3 = nn.Linear(32, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(-1, 32 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # x = F.relu(self.fc2_2(x))
        return self.fc3(x)


class RGBBCRobot2(RobotPolicy):
    def __init__(self):
        torch.manual_seed(42)
        super(RGBBCRobot2, self).__init__()
        self.policy = NBC()
        self.optimizer = optim.Adam(self.policy.parameters(), lr=0.0005,weight_decay=0.000001)
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    def train(self, data):

        observations = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2)  # (N, C, H, W)
        actions = torch.tensor(data['actions'], dtype=torch.long)

        dataset = TensorDataset(observations, actions)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True,num_workers=0,pin_memory=False)

        # print("start Trainning")
        for epoch in range(10):  # Training for 10 epochs
            for inputs, labels in dataloader:
                self.optimizer.zero_grad()
                outputs = self.policy(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

    def get_action(self, obs):
        obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2)  # (1, C, H, W)
        self.policy.eval()
        with torch.no_grad():
            outputs = self.policy(obs)
            _, predicted = torch.max(outputs, 1)
            action = predicted.item()
        return action

# # Implement your solution for Part 3 below
# #Part 3 Grade: 0.92 / 0.95 * 6 = 5.96
# #------------------------------------------------------------------------------------------
# # actions (12000,)
# # obs (12000, 64, 64, 3)
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset

# class NBC(nn.Module):
#     def __init__(self):
#         super(NBC, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(32 * 16 * 16,128)  # With 32 feature maps and each of size 16x16 (after two pooling operations),
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 4)

#         # self.fc2 = nn.Linear(128, 64)
#         # self.fc2_2 = nn.Linear(64, 32)
#         # self.fc3 = nn.Linear(32, 4)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.reshape(-1, 32 * 16 * 16)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))

#         # x = F.relu(self.fc2_2(x))
#         return self.fc3(x)


# class RGBBCRobot2(RobotPolicy):
#     def __init__(self):
#         torch.manual_seed(42)
#         super(RGBBCRobot2, self).__init__()
#         self.policy = NBC()
#         self.optimizer = optim.Adam(self.policy.parameters(), lr=0.0005,weight_decay=0.000001)
#         self.criterion = nn.CrossEntropyLoss()

#     def train(self, data):

#         observations = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2)  # (N, C, H, W)
#         actions = torch.tensor(data['actions'], dtype=torch.long)

#         dataset = TensorDataset(observations, actions)
#         dataloader = DataLoader(dataset, batch_size=64, shuffle=False,num_workers=6,pin_memory=False)

#         # print("start Trainning")
#         for epoch in range(2):  # Training for 9 epochs
#             for inputs, labels in dataloader:
#                 self.optimizer.zero_grad()
#                 outputs = self.policy(inputs)
#                 loss = self.criterion(outputs, labels)
#                 loss.backward()
#                 self.optimizer.step()
#             print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

#     def get_action(self, obs):
#         obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2)  # (1, C, H, W)
#         self.policy.eval()
#         with torch.no_grad():
#             outputs = self.policy(obs)
#             _, predicted = torch.max(outputs, 1)
#             action = predicted.item()
#         return action


def score(times):
  for i in range(times):
    start = time.time()
    part1_bound = 0.99
    part2_bound = 0.95
    part3_bound = 0.95
    score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(), gui=False, model=None)
    end = time.time()
    grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6

    print(f'Score: {score3}',f'Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}','it took',(end - start)/60,'min' )
    print('\n--------------------------------------------------------------------------------- ')
# score(2)

In [ ]:
# @title 5.96 ignore

# # Implement your solution for Part 3 below
# #Part 3 Grade: 0.92 / 0.95 * 6 = 5.96
# #------------------------------------------------------------------------------------------
# # actions (12000,)
# # obs (12000, 64, 64, 3)
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset

# class NBC(nn.Module):
#     def __init__(self):
#         super(NBC, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(32 * 16 * 16,128)  # With 32 feature maps and each of size 16x16 (after two pooling operations),
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 4)

#         # self.fc2 = nn.Linear(128, 64)
#         # self.fc2_2 = nn.Linear(64, 32)
#         # self.fc3 = nn.Linear(32, 4)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.reshape(-1, 32 * 16 * 16)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))

#         # x = F.relu(self.fc2_2(x))
#         return self.fc3(x)


# class RGBBCRobot2(RobotPolicy):
#     def __init__(self):
#         torch.manual_seed(42)
#         super(RGBBCRobot2, self).__init__()
#         self.policy = NBC()
#         self.optimizer = optim.Adam(self.policy.parameters(), lr=0.0005,weight_decay=0.000001)
#         self.criterion = nn.CrossEntropyLoss()

#     def train(self, data):

#         observations = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2)  # (N, C, H, W)
#         actions = torch.tensor(data['actions'], dtype=torch.long)

#         dataset = TensorDataset(observations, actions)
#         dataloader = DataLoader(dataset, batch_size=64, shuffle=False,num_workers=6,pin_memory=False)

#         print("start Trainning")
#         for epoch in range(9):  # Training for 9 epochs
#             for inputs, labels in dataloader:
#                 self.optimizer.zero_grad()
#                 outputs = self.policy(inputs)
#                 loss = self.criterion(outputs, labels)
#                 loss.backward()
#                 self.optimizer.step()
#             print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

#     def get_action(self, obs):
#         obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2)  # (1, C, H, W)
#         self.policy.eval()
#         with torch.no_grad():
#             outputs = self.policy(obs)
#             _, predicted = torch.max(outputs, 1)
#             action = predicted.item()
#         return action
# def score(times):
#   for i in range(times):
#     start = time.time()
#     part1_bound = 0.99
#     part2_bound = 0.95
#     part3_bound = 0.95
#     score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(), gui=False, model=None)
#     end = time.time()
#     grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6
#     print('it took',(end - start)/60,'min' )
#     print('\n---')
#     print(f'Part 3 Score: {score3}')
#     print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}')
#     print('\n--------------------------------------------------------------------------------- ')
# score(1)
# # @title 5.78 ignore
# # Implement your solution for Part 3 below
# #seed 12 Part 3 Grade: 0.92 / 0.95 * 6 = 5.78
# # actions (12000,)
# # obs (12000, 64, 64, 3)\
# import random
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset

# # class NBC(nn.Module):
# #     def __init__(self):
# #         super(NBC, self).__init__()
# #         self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
# #         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

# #         self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
# #         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

# #         self.fc1 = nn.Linear(32 * 16 * 16,128)  # With 32 feature maps and each of size 16x16 (after two pooling operations),
# #         self.fc2 = nn.Linear(128, 64)
# #         self.fc3 = nn.Linear(64, 4)

# #         # self.fc2 = nn.Linear(128, 64)
# #         # self.fc2_2 = nn.Linear(64, 32)
# #         # self.fc3 = nn.Linear(32, 4)

# #     def forward(self, x):
# #         x = self.pool(F.relu(self.conv1(x)))
# #         x = self.pool(F.relu(self.conv2(x)))
# #         x = x.reshape(-1, 32 * 16 * 16)
# #         x = F.relu(self.fc1(x))
# #         x = F.relu(self.fc2(x))
# #         # x = F.relu(self.fc2_2(x))
# #         return self.fc3(x)

# class NBC(nn.Module):
#     def __init__(self):
#         super(NBC, self).__init__()

#         # Define the convolutional part of the network
#         self.conv_layers = nn.Sequential(
#             nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         # Define the fully connected part of the network
#         self.fc_layers = nn.Sequential(
#             nn.Linear(32 * 16 * 16, 128),
#             nn.ReLU(),
#             nn.Linear(128, 80),
#             nn.ReLU(),
#             nn.Linear(80, 4)
#         )

#     def forward(self, x):
#         # Apply the convolutional layers
#         x = self.conv_layers(x)

#         # Flatten the output for the fully connected layers
#         x = x.reshape(-1, 32 * 16 * 16)

#         # Apply the fully connected layers
#         x = self.fc_layers(x)

#         return x
# class RGBBCRobot2(RobotPolicy):
#     def __init__(self,seed = 0):
#         torch.manual_seed(seed)
#         super(RGBBCRobot2, self).__init__()
#         self.policy = NBC()
#         self.optimizer = optim.Adam(self.policy.parameters(), lr=0.0005,weight_decay=0.00001,maximize=False)
#         self.criterion = nn.CrossEntropyLoss()

#     def train(self, data):

#         observations = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2)  # (N, C, H, W)
#         actions = torch.tensor(data['actions'], dtype=torch.long)
#         dataloader = DataLoader(TensorDataset(observations, actions), batch_size=82, shuffle=False,num_workers=6,pin_memory=False)

#         # print("start Trainning")
#         for epoch in range(10):  # Training for 10 epochs
#             for inputs, labels in dataloader:
#                 self.optimizer.zero_grad()
#                 outputs = self.policy(inputs)
#                 loss = self.criterion(outputs, labels)
#                 loss.backward()
#                 self.optimizer.step()
#             # print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

#     def get_action(self, obs):
#         obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2)  # (1, C, H, W)
#         self.policy.eval()
#         with torch.no_grad():
#             outputs = self.policy(obs)
#             _, predicted = torch.max(outputs, 1)
#             action = predicted.item()
#         return action

# import random

# def search(times):
#     best = 5
#     best_seed = int
#     for i in range(times):

#         start = time.time()
#         part1_bound = 0.99
#         part2_bound = 0.95
#         part3_bound = 0.95
#         score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(seed=i), gui=True, model=None)
#         end = time.time()
#         grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6
#         if grade3 >= best:
#             best = grade3
#             best_seed = i
#         if grade3 >= best:
#             print('it took',(end - start)/60,'min' )
#             print('\n---')
#             print(f'Part 3 Score: {score3}')
#             print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed: ",i)
#             print('\n--------------------------------------------------------------------------------- ')
#         print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed: ",i)
#     print('best score is ',best,'using seed: ',best_seed)

# def score(times):
#     best = 5
#     best_seed = int
#     for i in range(times):
#         start = time.time()
#         part1_bound = 0.99
#         part2_bound = 0.95
#         part3_bound = 0.95
#         score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(seed=12), gui=False, model=None)
#         end = time.time()
#         grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6
#         print('it took',(end - start)/60,'min' )
#         print('\n---')
#         print(f'Part 3 Score: {score3}')
#         print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed: ",i)
#         print('\n--------------------------------------------------------------------------------- ')
#         print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed: ",i)
#     print('best score is ',best,'using seed: ',best_seed)


# score(1)
# # @title 5.78 ignore
# # Implement your solution for Part 3 below
# #seed 12 Part 3 Grade: 0.92 / 0.95 * 6 = 5.78
# # actions (12000,)
# # obs (12000, 64, 64, 3)\
# import random
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# # class NBC(nn.Module):
# #     def __init__(self):
# #         super(NBC, self).__init__()
# #         self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
# #         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

# #         self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
# #         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

# #         self.fc1 = nn.Linear(32 * 16 * 16,128)  # With 32 feature maps and each of size 16x16 (after two pooling operations),
# #         self.fc2 = nn.Linear(128, 64)
# #         self.fc3 = nn.Linear(64, 4)

# #         # self.fc2 = nn.Linear(128, 64)
# #         # self.fc2_2 = nn.Linear(64, 32)
# #         # self.fc3 = nn.Linear(32, 4)

# #     def forward(self, x):
# #         x = self.pool(F.relu(self.conv1(x)))
# #         x = self.pool(F.relu(self.conv2(x)))
# #         x = x.reshape(-1, 32 * 16 * 16)
# #         x = F.relu(self.fc1(x))
# #         x = F.relu(self.fc2(x))
# #         # x = F.relu(self.fc2_2(x))
# #         return self.fc3(x)

# class NBC(nn.Module):
#     def __init__(self, upper_layers = 128 , lower_layers = 64 ):
#         super(NBC, self).__init__()

#         # Define the convolutional part of the network
#         self.conv_layers = nn.Sequential(
#             nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         # Define the fully connected part of the network
#         self.fc_layers = nn.Sequential(
#             nn.Linear(32 * 16 * 16, upper_layers),
#             nn.ReLU(),
#             nn.Linear(upper_layers, lower_layers),
#             nn.ReLU(),
#             nn.Linear(lower_layers, 4)
#         )

#     def forward(self, x):
#         # Apply the convolutional layers
#         x = self.conv_layers(x)

#         # Flatten the output for the fully connected layers
#         x = x.reshape(-1, 32 * 16 * 16)

#         # Apply the fully connected layers
#         x = self.fc_layers(x)

#         return x
# class RGBBCRobot2(RobotPolicy):
#     def __init__(self, seed=0, upper_layers=128, lower_layers=64):
#         self.seed = seed
#         torch.manual_seed(seed)
#         super(RGBBCRobot2, self).__init__()
#         self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Set device to CUDA if available
#         self.policy = NBC(upper_layers=upper_layers, lower_layers=lower_layers).to(self.device)  # Move the model to the specified device
#         self.optimizer = optim.Adam(self.policy.parameters(), lr=0.0005, weight_decay=0.00001, maximize=False)
#         self.criterion = nn.CrossEntropyLoss()

#     def train(self, data):
#         torch.manual_seed(self.seed)
#         observations = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2).to(self.device)  # Move data to the specified device
#         actions = torch.tensor(data['actions'], dtype=torch.long).to(self.device)
#         dataloader = DataLoader(TensorDataset(observations, actions), batch_size=50, shuffle=False)

#         for epoch in range(9):  # Training loop remains the same
#             for inputs, labels in dataloader:
#                 self.optimizer.zero_grad()
#                 outputs = self.policy(inputs)
#                 loss = self.criterion(outputs, labels)
#                 loss.backward()
#                 self.optimizer.step()

#     def get_action(self, obs):
#         obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2).to(self.device)  # Move data to the specified device
#         self.policy.eval()
#         with torch.no_grad():
#             outputs = self.policy(obs)
#             _, predicted = torch.max(outputs, 1)
#             action = predicted.item()
#         return action

# import random
# def score(times):
#     best = 5
#     best_seed = int
#     for i in range(times):
#         start = time.time()
#         part1_bound = 0.99
#         part2_bound = 0.95
#         part3_bound = 0.95
#         score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(seed=12), gui=False, model=None)
#         end = time.time()
#         grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6
#         print('it took',(end - start)/60,'min' )
#         print('\n---')
#         print(f'Part 3 Score: {score3}')
#         print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed: ",i)
#         print('\n--------------------------------------------------------------------------------- ')
#         print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed: ",i)
#     print('best score is ',best,'using seed: ',best_seed)


# def search(times):
#     part1_bound = 0.99
#     part2_bound = 0.95
#     part3_bound = 0.95
#     upper = random.randint(100, 200)
#     lower = random.randint(30, 100)

#     best_up = int
#     best_lower = int

#     best = 5
#     best_seed = int
#     road = []
#     for i in range(times):
#         # upper = random.randint(100, 200)
#         upper = 128
#         lower = i+50
#         start = time.time()
#         score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(
#             seed=42,
#             upper_layers = upper,
#             lower_layers = lower), gui=True, model=None)


#         end = time.time()
#         grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6
#         road.append(grade3)
#         if grade3 > 5.:
#             print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#             print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed:",seed ,' upper:',upper,' lower: ',lower,  'it took',(end - start)/60,'min' )
#             print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#         if grade3 >= best:
#             best = grade3
#             best_seed = i
#             best_up = upper
#             best_lower = lower
#         print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed:",42 ,' upper:',upper,' lower: ',lower,  'it took',(end - start)/60,'min' )
#     print('best score is ',best,'using seed: ',best_seed,' upper: ')
#     print(road)

# # Part 3 Grade: 0.94 / 0.95 * 6 = 5.95 using seed: 42  upper: 128  lower:  72 it took 0.22193415562311808 min
# #  http://127.0.0.1:9000/?token=41babf1b4989a96c20b487e47be174c092471678d139cd95
# #best score is  5.645634918919605 using seed:  1
# search(128)
# # Part 3 Grade: 0.89 / 0.95 * 6 = 5.62 using seed: 42  upper: 128  lower:  50 it took 0.409449028968811 min
# # Part 3 Grade: 0.79 / 0.95 * 6 = 5.01 using seed: 42  upper: 128  lower:  51 it took 0.46328362623850505 min
# # Part 3 Grade: 0.89 / 0.95 * 6 = 5.64 using seed: 42  upper: 128  lower:  52 it took 0.40527234474817914 min
# # Part 3 Grade: 0.85 / 0.95 * 6 = 5.36 using seed: 42  upper: 128  lower:  53 it took 0.43886736234029133 min
# # Part 3 Grade: 0.78 / 0.95 * 6 = 4.92 using seed: 42  upper: 128  lower:  54 it took 0.4735151489575704 min
# # Part 3 Grade: 0.84 / 0.95 * 6 = 5.31 using seed: 42  upper: 128  lower:  55 it took 0.4221602956453959 min
# # Part 3 Grade: 0.78 / 0.95 * 6 = 4.96 using seed: 42  upper: 128  lower:  56 it took 0.42792600790659585 min
# # Part 3 Grade: 0.88 / 0.95 * 6 = 5.54 using seed: 42  upper: 128  lower:  57 it took 0.4391207615534464 min
# # Part 3 Grade: 0.82 / 0.95 * 6 = 5.20 using seed: 42  upper: 128  lower:  58 it took 0.4360662261644999 min
# # Part 3 Grade: 0.89 / 0.95 * 6 = 5.60 using seed: 42  upper: 128  lower:  59 it took 0.41809734106063845 min
# # Part 3 Grade: 0.82 / 0.95 * 6 = 5.20 using seed: 42  upper: 128  lower:  60 it took 0.44941208362579343 min
# # Part 3 Grade: 0.78 / 0.95 * 6 = 4.90 using seed: 42  upper: 128  lower:  61 it took 0.5071820894877116 min
# # Part 3 Grade: 0.81 / 0.95 * 6 = 5.10 using seed: 42  upper: 128  lower:  62 it took 0.4757840832074483 min
# # Part 3 Grade: 0.82 / 0.95 * 6 = 5.17 using seed: 42  upper: 128  lower:  63 it took 0.4459131956100464 min
# # Part 3 Grade: 0.85 / 0.95 * 6 = 5.39 using seed: 42  upper: 128  lower:  64 it took 0.4897317886352539 min
# # Part 3 Grade: 0.84 / 0.95 * 6 = 5.28 using seed: 42  upper: 128  lower:  65 it took 0.474860405921936 min
# # Part 3 Grade: 0.84 / 0.95 * 6 = 5.29 using seed: 42  upper: 128  lower:  66 it took 0.46219141483306886 min
# # Part 3 Grade: 0.85 / 0.95 * 6 = 5.35 using seed: 42  upper: 128  lower:  67 it took 0.4184943517049154 min
# # Part 3 Grade: 0.80 / 0.95 * 6 = 5.04 using seed: 42  upper: 128  lower:  68 it took 0.4725057641665141 min
# # Part 3 Grade: 0.86 / 0.95 * 6 = 5.41 using seed: 42  upper: 128  lower:  69 it took 0.43774845202763873 min
# # Part 3 Grade: 0.82 / 0.95 * 6 = 5.19 using seed: 42  upper: 128  lower:  70 it took 0.4664644956588745 min
# # Part 3 Grade: 0.82 / 0.95 * 6 = 5.17 using seed: 42  upper: 128  lower:  71 it took 0.4663043657938639 min
# # Part 3 Grade: 0.87 / 0.95 * 6 = 5.49 using seed: 42  upper: 128  lower:  72 it took 0.4655236005783081 min
# # Part 3 Grade: 0.90 / 0.95 * 6 = 5.68 using seed: 42  upper: 128  lower:  73 it took 0.40431360801060995 min
# # Part 3 Grade: 0.90 / 0.95 * 6 = 5.71 using seed: 42  upper: 128  lower:  74 it took 0.4144969979921977 min
# # Part 3 Grade: 0.88 / 0.95 * 6 = 5.53 using seed: 42  upper: 128  lower:  75 it took 0.4205396016438802 min
# # Part 3 Grade: 0.89 / 0.95 * 6 = 5.63 using seed: 42  upper: 128  lower:  76 it took 0.4014447212219238 min
# # Part 3 Grade: 0.84 / 0.95 * 6 = 5.30 using seed: 42  upper: 128  lower:  77 it took 0.44132861296335857 min
# # Part 3 Grade: 0.84 / 0.95 * 6 = 5.33 using seed: 42  upper: 128  lower:  78 it took 0.40471431414286296 min
# # Part 3 Grade: 0.87 / 0.95 * 6 = 5.51 using seed: 42  upper: 128  lower:  79 it took 0.42035337289174396 min
# # Part 3 Grade: 0.90 / 0.95 * 6 = 5.66 using seed: 42  upper: 128  lower:  80 it took 0.4022068023681641 min
# # Part 3 Grade: 0.87 / 0.95 * 6 = 5.49 using seed: 42  upper: 128  lower:  81 it took 0.3736559748649597 min
# # Part 3 Grade: 0.85 / 0.95 * 6 = 5.38 using seed: 42  upper: 128  lower:  82 it took 0.5035714030265808 min
# # Part 3 Grade: 0.91 / 0.95 * 6 = 5.77 using seed: 42  upper: 128  lower:  83 it took 0.4765840093294779 min
# # Part 3 Grade: 0.82 / 0.95 * 6 = 5.16 using seed: 42  upper: 128  lower:  84 it took 0.602639126777649 min
# # Part 3 Grade: 0.83 / 0.95 * 6 = 5.25 using seed: 42  upper: 128  lower:  85 it took 0.5524603764216105 min
# # Part 3 Grade: 0.89 / 0.95 * 6 = 5.65 using seed: 42  upper: 128  lower:  86 it took 0.5460824807484944 min
# # Part 3 Grade: 0.85 / 0.95 * 6 = 5.34 using seed: 42  upper: 128  lower:  87 it took 0.5482121268908183 min
# # Part 3 Grade: 0.82 / 0.95 * 6 = 5.21 using seed: 42  upper: 128  lower:  88 it took 0.5984585245450338 min
# # Part 3 Grade: 0.91 / 0.95 * 6 = 5.72 using seed: 42  upper: 128  lower:  89 it took 0.5158032933870952 min
# # Part 3 Grade: 0.86 / 0.95 * 6 = 5.44 using seed: 42  upper: 128  lower:  90 it took 0.5568669517834981 min
# # Part 3 Grade: 0.91 / 0.95 * 6 = 5.72 using seed: 42  upper: 128  lower:  91 it took 0.48815372387568157 min
# # Part 3 Grade: 0.86 / 0.95 * 6 = 5.42 using seed: 42  upper: 128  lower:  92 it took 0.5320407112439474 min
# # Part 3 Grade: 0.81 / 0.95 * 6 = 5.12 using seed: 42  upper: 128  lower:  93 it took 0.6255919337272644 min
# # Part 3 Grade: 0.84 / 0.95 * 6 = 5.27 using seed: 42  upper: 128  lower:  94 it took 0.5439372340838114 min
# # Part 3 Grade: 0.82 / 0.95 * 6 = 5.15 using seed: 42  upper: 128  lower:  95 it took 0.5375640233357747 min
# # Part 3 Grade: 0.81 / 0.95 * 6 = 5.11 using seed: 42  upper: 128  lower:  96 it took 0.5329799294471741 min
# # Part 3 Grade: 0.81 / 0.95 * 6 = 5.14 using seed: 42  upper: 128  lower:  97 it took 0.550384255250295 min
# # Part 3 Grade: 0.80 / 0.95 * 6 = 5.06 using seed: 42  upper: 128  lower:  98 it took 0.5128353794415792 min
# # Part 3 Grade: 0.80 / 0.95 * 6 = 5.04 using seed: 42  upper: 128  lower:  99 it took 0.5332785805066427 min
# # Part 3 Grade: 0.83 / 0.95 * 6 = 5.24 using seed: 42  upper: 128  lower:  100 it took 0.5263606667518616 min
# # Part 3 Grade: 0.84 / 0.95 * 6 = 5.28 using seed: 42  upper: 128  lower:  101 it took 0.5028890411059062 min
# # Part 3 Grade: 0.86 / 0.95 * 6 = 5.41 using seed: 42  upper: 128  lower:  102 it took 0.5560908635457357 min
# # Part 3 Grade: 0.88 / 0.95 * 6 = 5.59 using seed: 42  upper: 128  lower:  103 it took 0.5500704209009807 min
# # Part 3 Grade: 0.87 / 0.95 * 6 = 5.49 using seed: 42  upper: 128  lower:  104 it took 0.5719008763631185 min
# # Part 3 Grade: 0.90 / 0.95 * 6 = 5.66 using seed: 42  upper: 128  lower:  105 it took 0.5306296706199646 min
# # Part 3 Grade: 0.91 / 0.95 * 6 = 5.77 using seed: 42  upper: 128  lower:  106 it took 0.49418185154596966 min
# # Part 3 Grade: 0.81 / 0.95 * 6 = 5.12 using seed: 42  upper: 128  lower:  107 it took 0.5134753187497457 min
# # Part 3 Grade: 0.86 / 0.95 * 6 = 5.46 using seed: 42  upper: 128  lower:  108 it took 0.48535792032877606 min
# # Part 3 Grade: 0.89 / 0.95 * 6 = 5.62 using seed: 42  upper: 128  lower:  109 it took 0.4783740599950155 min
# # Part 3 Grade: 0.84 / 0.95 * 6 = 5.31 using seed: 42  upper: 128  lower:  110 it took 0.5042753100395203 min
# # Part 3 Grade: 0.82 / 0.95 * 6 = 5.19 using seed: 42  upper: 128  lower:  111 it took 0.5049405852953593 min
# # Part 3 Grade: 0.76 / 0.95 * 6 = 4.83 using seed: 42  upper: 128  lower:  112 it took 0.5241187890370687 min
# # Part 3 Grade: 0.75 / 0.95 * 6 = 4.71 using seed: 42  upper: 128  lower:  113 it took 0.49179720481236777 min
# # Part 3 Grade: 0.88 / 0.95 * 6 = 5.53 using seed: 42  upper: 128  lower:  114 it took 0.42627280950546265 min
# # Part 3 Grade: 0.85 / 0.95 * 6 = 5.37 using seed: 42  upper: 128  lower:  115 it took 0.4382785598436991 min
# # Part 3 Grade: 0.86 / 0.95 * 6 = 5.41 using seed: 42  upper: 128  lower:  116 it took 0.410961377620697 min
# # @title  5.78 ignore
# # Implement your solution for Part 3 below
# #seed 12 Part 3 Grade: 0.92 / 0.95 * 6 = 5.78
# # actions (12000,)
# # obs (12000, 64, 64, 3)\
# import random
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# # class NBC(nn.Module):
# #     def __init__(self):
# #         super(NBC, self).__init__()
# #         self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
# #         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

# #         self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
# #         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

# #         self.fc1 = nn.Linear(32 * 16 * 16,128)  # With 32 feature maps and each of size 16x16 (after two pooling operations),
# #         self.fc2 = nn.Linear(128, 64)
# #         self.fc3 = nn.Linear(64, 4)

# #         # self.fc2 = nn.Linear(128, 64)
# #         # self.fc2_2 = nn.Linear(64, 32)
# #         # self.fc3 = nn.Linear(32, 4)

# #     def forward(self, x):
# #         x = self.pool(F.relu(self.conv1(x)))
# #         x = self.pool(F.relu(self.conv2(x)))
# #         x = x.reshape(-1, 32 * 16 * 16)
# #         x = F.relu(self.fc1(x))
# #         x = F.relu(self.fc2(x))
# #         # x = F.relu(self.fc2_2(x))
# #         return self.fc3(x)

# class NBC(nn.Module):
#     def __init__(self, upper_layers = 128 , lower_layers = 64 ):
#         super(NBC, self).__init__()

#         # Define the convolutional part of the network
#         self.conv_layers = nn.Sequential(
#             nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         # Define the fully connected part of the network
#         self.fc_layers = nn.Sequential(
#             nn.Linear(32 * 16 * 16, upper_layers),
#             nn.ReLU(),
#             nn.Linear(upper_layers, lower_layers),
#             nn.ReLU(),
#             nn.Linear(lower_layers, 4)
#         )

#     def forward(self, x):
#         # Apply the convolutional layers
#         x = self.conv_layers(x)

#         # Flatten the output for the fully connected layers
#         x = x.reshape(-1, 32 * 16 * 16)

#         # Apply the fully connected layers
#         x = self.fc_layers(x)

#         return x
# class RGBBCRobot2(RobotPolicy):
#     def __init__(self, seed=0, upper_layers=128, lower_layers=64):
#         self.seed = seed
#         torch.manual_seed(seed)
#         super(RGBBCRobot2, self).__init__()
#         self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Set device to CUDA if available
#         self.policy = NBC(upper_layers=upper_layers, lower_layers=lower_layers).to(self.device)  # Move the model to the specified device
#         self.optimizer = optim.Adam(self.policy.parameters(), lr=0.0005, maximize=False, weight_decay=0.00001)#, weight_decay=0.00001
#         self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

#     def train(self, data):
#         torch.manual_seed(self.seed)
#         observations = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2).to(self.device)  # Move data to the specified device
#         actions = torch.tensor(data['actions'], dtype=torch.long).to(self.device)
#         dataloader = DataLoader(TensorDataset(observations, actions), batch_size=50, shuffle=False)

#         for epoch in range(9):  # Training loop remains the same
#             for inputs, labels in dataloader:
#                 self.optimizer.zero_grad()
#                 outputs = self.policy(inputs)
#                 loss = self.criterion(outputs, labels)
#                 loss.backward()
#                 self.optimizer.step()

#     def get_action(self, obs):
#         obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2).to(self.device)  # Move data to the specified device
#         self.policy.eval()
#         with torch.no_grad():
#             outputs = self.policy(obs)
#             _, predicted = torch.max(outputs, 1)
#             action = predicted.item()
#         return action

# import random
# def score(times):
#     best = 5
#     best_seed = int
#     for i in range(times):
#         start = time.time()
#         part1_bound = 0.99
#         part2_bound = 0.95
#         part3_bound = 0.95
#         score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(seed=12), gui=False, model=None)
#         end = time.time()
#         grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6
#         print('it took',(end - start)/60,'min' )
#         print('\n---')
#         print(f'Part 3 Score: {score3}')
#         print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed: ",i)
#         print('\n--------------------------------------------------------------------------------- ')
#         print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed: ",i)
#     print('best score is ',best,'using seed: ',best_seed)


# def search(times):
#     part1_bound = 0.99
#     part2_bound = 0.95
#     part3_bound = 0.95
#     upper = random.randint(100, 200)
#     lower = random.randint(30, 100)

#     best_up = int
#     best_lower = int

#     best = 5
#     best_seed = int
#     road = []
#     for i in range(times):
#         upper = random.randint(100, 200)
#         # upper = 128
#         lower = random.randint(54, 128)
#         start = time.time()
#         score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(
#             seed=42,
#             upper_layers = upper,
#             lower_layers = lower), gui=True, model=None)


#         end = time.time()
#         grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6
#         road.append(grade3)
#         print('Score:',score3,f'Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed:",42 ,' upper:',upper,' lower: ',lower,  'it took',(end - start)/60,'min',' ',i )
#         if grade3 > 5.9:
#             print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#             print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed:" ,' upper:',upper,' lower: ',lower,  'it took',(end - start)/60,'min' )
#             print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#         if grade3 >= best:
#             best = grade3
#             best_seed = i
#             best_up = upper
#             best_lower = lower
#         # print('Score:',score3,f'Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}',"using seed:",42 ,' upper:',upper,' lower: ',lower,  'it took',(end - start)/60,'min'  )
#     print('best score is ',best,'using seed: ',best_seed,' upper: ')
#     print(road)

# #  http://127.0.0.1:9000/?token=41babf1b4989a96c20b487e47be174c092471678d139cd95
# #best score is  5.645634918919605 using seed:  1
# # 0    Score: 0.8729912826595501 Grade: 0.87 / 0.95 * 6 = 5.51 using seed: 42  upper: 194  lower:  95 it took 0.3906903902689616 min
# # 1    Score: 0.7934503859172121 Grade: 0.79 / 0.95 * 6 = 5.01 using seed: 42  upper: 104  lower:  105 it took 0.4322916110356649 min
# # 2    Score: 0.8535724810780424 Grade: 0.85 / 0.95 * 6 = 5.39 using seed: 42  upper: 195  lower:  119 it took 0.40486913522084556 min
# # 3    Score: 0.8530529807631685 Grade: 0.85 / 0.95 * 6 = 5.39 using seed: 42  upper: 168  lower:  119 it took 0.4156123598416646 min
# # 4    Score: 0.8380412113077013 Grade: 0.84 / 0.95 * 6 = 5.29 using seed: 42  upper: 198  lower:  100 it took 0.46597175995508827 min
# # 5    Score: 0.804060831555783 Grade: 0.80 / 0.95 * 6 = 5.08 using seed: 42  upper: 111  lower:  69 it took 0.4877118547757467 min
# # 6    Score: 0.8626262000102295 Grade: 0.86 / 0.95 * 6 = 5.45 using seed: 42  upper: 140  lower:  82 it took 0.43262551228205365 min
# # 7    Score: 0.9101472471451185 Grade: 0.91 / 0.95 * 6 = 5.75 using seed: 42  upper: 106  lower:  124 it took 0.40864394505818685 min
# # 8    Score: 0.8108696559148608 Grade: 0.81 / 0.95 * 6 = 5.12 using seed: 42  upper: 195  lower:  111 it took 0.4483546853065491 min
# # 9    Score: 0.8978145151916592 Grade: 0.90 / 0.95 * 6 = 5.67 using seed: 42  upper: 159  lower:  107 it took 0.4124769886334737 min
# # 10    Score: 0.8474023426792939 Grade: 0.85 / 0.95 * 6 = 5.35 using seed: 42  upper: 161  lower:  84 it took 0.4171638607978821 min
# # 11    Score: 0.8388844065082934 Grade: 0.84 / 0.95 * 6 = 5.30 using seed: 42  upper: 130  lower:  78 it took 0.44696821371714274 min
# # 12    Score: 0.8398258467477302 Grade: 0.84 / 0.95 * 6 = 5.30 using seed: 42  upper: 178  lower:  95 it took 0.4307393630345662 min
# # 13    Score: 0.8435482998858547 Grade: 0.84 / 0.95 * 6 = 5.33 using seed: 42  upper: 135  lower:  116 it took 0.4326540191968282 min
# # 14    Score: 0.8526144847629327 Grade: 0.85 / 0.95 * 6 = 5.38 using seed: 42  upper: 101  lower:  119 it took 0.4567560195922852 min
# # 15    Score: 0.814618357934618 Grade: 0.81 / 0.95 * 6 = 5.14 using seed: 42  upper: 104  lower:  127 it took 0.4593956430753072 min
# # 16    Score: 0.7828437673509775 Grade: 0.78 / 0.95 * 6 = 4.94 using seed: 42  upper: 103  lower:  110 it took 0.46786193052927655 min
# # 17    Score: 0.7645751582666616 Grade: 0.76 / 0.95 * 6 = 4.83 using seed: 42  upper: 108  lower:  88 it took 0.485747758547465 min
# # 18    Score: 0.921488229132095 Grade: 0.92 / 0.95 * 6 = 5.82 using seed: 42  upper: 195  lower:  97 it took 0.38218544324239095 min
# # 19    Score: 0.8645566781355317 Grade: 0.86 / 0.95 * 6 = 5.46 using seed: 42  upper: 181  lower:  82 it took 0.4263224720954895 min
# # 20    Score: 0.8443751424760889 Grade: 0.84 / 0.95 * 6 = 5.33 using seed: 42  upper: 117  lower:  106 it took 0.4326399087905884 min
# # 21    Score: 0.7980353701014089 Grade: 0.80 / 0.95 * 6 = 5.04 using seed: 42  upper: 143  lower:  89 it took 0.458575963973999 min
# # 22    Score: 0.8265725815892467 Grade: 0.83 / 0.95 * 6 = 5.22 using seed: 42  upper: 196  lower:  96 it took 0.4200365702311198 min
# # 23    Score: 0.780361355497546 Grade: 0.78 / 0.95 * 6 = 4.93 using seed: 42  upper: 189  lower:  111 it took 0.47960570255915325 min
# # 24    Score: 0.824604878653716 Grade: 0.82 / 0.95 * 6 = 5.21 using seed: 42  upper: 105  lower:  56 it took 0.43912495374679567 min
# # 25    Score: 0.7573545853973261 Grade: 0.76 / 0.95 * 6 = 4.78 using seed: 42  upper: 193  lower:  106 it took 0.49390085935592654 min
# # 26    Score: 0.8499595282857472 Grade: 0.85 / 0.95 * 6 = 5.37 using seed: 42  upper: 198  lower:  117 it took 0.4216253717740377 min
# # 27    Score: 0.8117211045278694 Grade: 0.81 / 0.95 * 6 = 5.13 using seed: 42  upper: 115  lower:  124 it took 0.48709630568822226 min
# # 28    Score: 0.8756210753221544 Grade: 0.88 / 0.95 * 6 = 5.53 using seed: 42  upper: 116  lower:  107 it took 0.417417315642039 min
# # 29    Score: 0.8799999137885983 Grade: 0.88 / 0.95 * 6 = 5.56 using seed: 42  upper: 148  lower:  74 it took 0.3985628128051758 min
# # 30    Score: 0.8394973706848012 Grade: 0.84 / 0.95 * 6 = 5.30 using seed: 42  upper: 116  lower:  58 it took 0.4311057289441427 min
# # 31    Score: 0.7541756825271188 Grade: 0.75 / 0.95 * 6 = 4.76 using seed: 42  upper: 185  lower:  70 it took 0.4751457810401917 min
# # 32    Score: 0.8101441476686461 Grade: 0.81 / 0.95 * 6 = 5.12 using seed: 42  upper: 137  lower:  60 it took 0.45881512959798176 min
# # 33    Score: 0.8207360081660817 Grade: 0.82 / 0.95 * 6 = 5.18 using seed: 42  upper: 193  lower:  63 it took 0.42544259627660114 min
# # 34    Score: 0.8587287143812271 Grade: 0.86 / 0.95 * 6 = 5.42 using seed: 42  upper: 187  lower:  102 it took 0.4373493989308675 min
# # 35    Score: 0.8318381676902077 Grade: 0.83 / 0.95 * 6 = 5.25 using seed: 42  upper: 113  lower:  78 it took 0.43053975105285647 min
# # 36    Score: 0.8385187669589178 Grade: 0.84 / 0.95 * 6 = 5.30 using seed: 42  upper: 160  lower:  68 it took 0.41132450898488365 min
# # 37    Score: 0.8506998135287819 Grade: 0.85 / 0.95 * 6 = 5.37 using seed: 42  upper: 132  lower:  60 it took 0.43809274832407635 min
# # 38    Score: 0.8443431601969976 Grade: 0.84 / 0.95 * 6 = 5.33 using seed: 42  upper: 174  lower:  67 it took 0.4123456358909607 min
# # 39    Score: 0.8292172126833792 Grade: 0.83 / 0.95 * 6 = 5.24 using seed: 42  upper: 134  lower:  66 it took 0.45855441093444826 min
# # 40    Score: 0.8143069760062525 Grade: 0.81 / 0.95 * 6 = 5.14 using seed: 42  upper: 116  lower:  106 it took 0.45734779437383016 min
# # 41    Score: 0.8396321862301005 Grade: 0.84 / 0.95 * 6 = 5.30 using seed: 42  upper: 157  lower:  86 it took 0.44052687486012776 min
# # 42    Score: 0.8342116880850208 Grade: 0.83 / 0.95 * 6 = 5.27 using seed: 42  upper: 191  lower:  71 it took 0.39676424662272136 min
# # 43    Score: 0.9054198294661002 Grade: 0.91 / 0.95 * 6 = 5.72 using seed: 42  upper: 194  lower:  59 it took 0.3896673878033956 min
# # 44    Score: 0.7558531107214295 Grade: 0.76 / 0.95 * 6 = 4.77 using seed: 42  upper: 178  lower:  107 it took 0.4701403737068176 min
# # 45    Score: 0.82779343781339 Grade: 0.83 / 0.95 * 6 = 5.23 using seed: 42  upper: 109  lower:  58 it took 0.42122944990793865 min
# # 46    Score: 0.9295291171216844 Grade: 0.93 / 0.95 * 6 = 5.87 using seed: 42  upper: 179  lower:  77 it took 0.42634766101837157 min
# # 47    Score: 0.8682798810869835 Grade: 0.87 / 0.95 * 6 = 5.48 using seed: 42  upper: 113  lower:  116 it took 0.4671875834465027 min
# # 48    Score: 0.8370293485797887 Grade: 0.84 / 0.95 * 6 = 5.29 using seed: 42  upper: 200  lower:  89 it took 0.5108375787734986 min
# # 49    Score: 0.8391739164367498 Grade: 0.84 / 0.95 * 6 = 5.30 using seed: 42  upper: 178  lower:  70 it took 0.5071077148119608 min
# # 50    Score: 0.8383732089910757 Grade: 0.84 / 0.95 * 6 = 5.29 using seed: 42  upper: 120  lower:  63 it took 0.49754981994628905 min
# search(1000)

# # Score: 0.9484545718499507 Grade: 0.95 / 0.95 * 6 = 5.99 using seed: 42  upper: 139  lower:  90 it took 0.4512041290601095 min   39
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# # Part 3 Grade: 0.95 / 0.95 * 6 = 5.99 using seed:  upper: 139  lower:  90 it took 0.4512041290601095 min
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# # Score: 0.8967015175173504 Grade: 0.90 / 0.95 * 6 = 5.66 using seed: 42  upper: 155  lower:  89 it took 0.47109071413675946 min   40

# # Score: 0.959725416610936 Grade: 0.96 / 0.95 * 6 = 6.00 using seed: 42  upper: 168  lower:  78 it took 0.3713714798291524 min   92
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# # Part 3 Grade: 0.96 / 0.95 * 6 = 6.00 using seed:  upper: 168  lower:  78 it took 0.3713714798291524 min
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# # Score: 0.858315406780804 Grade: 0.86 / 0.95 * 6 = 5.42 using seed: 42  upper: 136  lower:  63 it took 0.4164063215255737 min   93

# # Score: 0.9689579876142039 Grade: 0.97 / 0.95 * 6 = 6.00 using seed: 42  upper: 114  lower:  117 it took 0.4121198534965515 min   181
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# # Part 3 Grade: 0.97 / 0.95 * 6 = 6.00 using seed:  upper: 114  lower:  117 it took 0.4121198534965515 min
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!YOU ARE GENIES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# # Score: 0.8295201959679452 Grade: 0.83 / 0.95 * 6 = 5.24 using seed: 42  upper: 168  lower:  71 it took 0.4873317797978719 min   182

## Evaluation and Grading


In [ ]:
# DO NOT CHANGE
# Getting the score and grade for Part 3

score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(), gui=False, model=None)
grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6

print('\n---')
print(f'Part 3 Score: {score3}')
print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}')

Epoch 1, Loss: 1.2335708141326904
Epoch 2, Loss: 1.206181287765503
Epoch 3, Loss: 0.6445888876914978
Epoch 4, Loss: 0.6723036766052246
Epoch 5, Loss: 0.5793219804763794
Epoch 6, Loss: 0.5745296478271484
Epoch 7, Loss: 0.552627444267273
Epoch 8, Loss: 0.5074701309204102
Epoch 9, Loss: 0.49692288041114807
Epoch 10, Loss: 0.4336121082305908

---
Part 3 Score: 0.9525130343849173
Part 3 Grade: 0.95 / 0.95 * 6 = 6.00


In [ ]:
# Optionally, uncomment and run the code below if you have saved an animation (gui = True) that you want to visualize.

# Image(filename='part_3_anim.png', width=200, height=200)

# Other Requirements and Hints

- **Training time**: To keep auto-grading feasible, your total training time must be strictly under 3 mins, 15mins, and 10 mins for parts 1, 2, and 3. These time budgets are more than enough to achieve full credits on this project. Note that longer training time does not necessarily mean higher performance because of overfitting. The faster your network trains the better!
- **Memory usage**: Make sure your code does not require too much memory. The required amount of RAM for this assignment should not be more than 8GB.
- **NO GPU**: No GPU is required or allowed for this assignment.
- **Reproducibility**: We have ensured that the randomness of the environment is deterministic. To get reproducible scores you must ensure your model training and prediction are also reproducible. The randomly initialized weights of the neural network should be made repeatable using seeding. You can add PyTorch seeding method below and see [PyTorch Reproducibility](https://pytorch.org/docs/stable/notes/randomness.html) to learn more.
  ```
  import torch
  torch.manual_seed(0)
  ```
- **Classifier**: In all the parts we are training a neural network to solve a classification problem and it is important to use a reasonable loss function. For example, the MSE (mean squared classification) error has drawbacks related to sensitivity. Cross entropy loss usually has good performance for classification tasks and you can find the documentation for it [here](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) and is further explained below. However, note that you are free to use any loss function you like.
  - Cross entropy is a concept from information theory which is defined for two probability distributions. Cross entropy is minimum when the two distributions involved are the same and this is the property that makes it useful as a loss function in the context of machine learning. The idea is to minimize the cross entropy between the prediction distribution and the label distribution. For our case where we are training a neural network for classification, we can have the network output a score for each action. Cross entropy can be computed from these scores by converting to probability values (using softmax) and comparing it with the label distribution. The label distribution is obtained simply by assigning a probability of 1 to ground truth action and 0 to all other actions. Once trained, the best action can found by just choosing the action with the highest probability (i.e., the highest score) as predicted by the network.
- **Optimizer**: While it is possible to use a simple optimizer to achieve the desired accuracy, the training time can be quite high. There exist a number of optimizers implemented in PyTorch that have much faster convergence.
- **Parameter tuning**: Keep your architectures simple and slowly add complexity (more layers/kernels) to improve accuracy. Remember "To Err is Human" and the expert data (collected by a human) that you are training on is not perfect. Having a 100% training accuracy (very small training loss) might not be the best for achieving the highest score. So make sure your model does not overfit during training.
- **PyTorch input shape**: Notice that the expected input shape to CONV2D in PyTorch is (N, C, H, W), where N is the batch size, C is the number of channels, H is the image height and W is the image width. You will need to switch axes for the incoming images in order for them to be correctly passed to the first convolution layer.